In [42]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from urllib.request import urlopen
import json
import geojson
import numpy as np

In [43]:
electricity_df = pd.read_csv('population.csv', sep = ',')
electricity_df.head()

,place,pop2023,growthRate,area,country,cca3,cca2,ccn3,region,subregion,landAreaKm,density,densityMi,Rank,rank
0,446,704149.0,0.01292,32.90,Macau,MAC,MO,446,Asia,Eastern Asia,32.9,21402.7052,55433.0064,167,1
1,492,36297.0,-0.00472,2.02,Monaco,MCO,MC,492,Europe,Western Europe,2.0,18148.5000,47004.6150,217,2
2,702,6014723.0,0.00653,710.00,Singapore,SGP,SG,702,Asia,South-Eastern Asia,718.0,8377.0515,21696.5635,114,3
3,344,7491609.0,0.00037,1104.00,Hong Kong,HKG,HK,344,Asia,Eastern Asia,1050.0,7134.8657,18479.3022,104,4
4,292,32688.0,0.00120,6.80,Gibraltar,GIB,GI,292,Europe,Southern Europe,6.8,4807.0588,12450.2824,219,5


In [44]:
filtered_df = electricity_df[['cca3', 'country', 'pop2023', 'density']]
filtered_df

,cca3,country,pop2023,density
0,MAC,Macau,704149.0,21402.7052
1,MCO,Monaco,36297.0,18148.5000
2,SGP,Singapore,6014723.0,8377.0515
3,HKG,Hong Kong,7491609.0,7134.8657
4,GIB,Gibraltar,32688.0,4807.0588
...,...,...,...,...
229,NAM,Namibia,2604172.0,3.1631
230,MNG,Mongolia,3447157.0,2.2133
231,ESH,Western Sahara,587259.0,2.2077
232,FLK,Falkland Islands,3791.0,0.3114


In [45]:
world = json.load(open('countries.geojson', 'r'))

In [46]:
import geopandas as gpd

see_countries = gpd.read_file('countries.geojson')


In [47]:

see_countries[see_countries['ISO_A3'] == 'TZA']

,ADMIN,ISO_A3,geometry
233,United Republic of Tanzania,TZA,"MULTIPOLYGON (((39.74916 -8.06549, 39.71944 -8..."


In [48]:
# MKD = Macedonia (148)
# SRB = Republic of Serbia (210)
# COD = Democratic Republic of the Congo	 (47)
# USA = United States of America (238)
# COG = Republic of Congo (48)



In [49]:
filtered_df.replace(to_replace= 'North Macedonia', value = 'Macedonia', inplace=True )
filtered_df.replace(to_replace= 'Serbia', value = 'Republic of Serbia', inplace=True )
filtered_df.replace(to_replace= 'DR Congo', value = 'Democratic Republic of the Congo', inplace=True )
filtered_df.replace(to_replace= 'United States', value = 'United States of America', inplace=True )
filtered_df.replace(to_replace= 'Republic of the Congo', value = 'Republic of Congo', inplace=True )
filtered_df.replace(to_replace= 'Tanzania', value = 'United Republic of Tanzania', inplace=True )

c:\Users\anaku\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5238: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
world['features'][0]['properties']

{'ADMIN': 'Aruba', 'ISO_A3': 'ABW'}

In [51]:
country_id_map = {}

for feature in world['features']:
    feature['id'] = feature['properties']['ADMIN']
    country_id_map[feature['properties']['ADMIN']] = feature['id']

In [52]:
print(filtered_df.index[filtered_df['country'] == 'United Republic of Tanzania'])

Int64Index([139], dtype='int64')


In [53]:
to_drop = [3, 11, 0, 14, 31, 35, 42, 58, 75, 82, 123, 138, 145, 174, 226]

In [54]:
filtered_df = filtered_df.drop(to_drop)

In [55]:
filtered_df['id'] = filtered_df['country'].apply(lambda x:country_id_map[x])

In [56]:
filtered_df

,cca3,country,pop2023,density,id
1,MCO,Monaco,36297.0,18148.5000,Monaco
2,SGP,Singapore,6014723.0,8377.0515,Singapore
4,GIB,Gibraltar,32688.0,4807.0588,Gibraltar
5,BHR,Bahrain,1485509.0,1892.3682,Bahrain
6,MDV,Maldives,521021.0,1736.7367,Maldives
...,...,...,...,...,...
229,NAM,Namibia,2604172.0,3.1631,Namibia
230,MNG,Mongolia,3447157.0,2.2133,Mongolia
231,ESH,Western Sahara,587259.0,2.2077,Western Sahara
232,FLK,Falkland Islands,3791.0,0.3114,Falkland Islands


In [57]:
filtered_df['Density scale'] = np.log10(filtered_df['density'])
# filtered_df

In [ ]:
fig = px.choropleth(
    filtered_df, 
    locations='id', 
    geojson=world, 
    color='Density scale',
    hover_name='country',
    hover_data=['density'])

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="White",
)

fig.show()

In [ ]:
fig = px.choropleth_mapbox(
    filtered_df, 
    locations='id', 
    geojson=world, 
    color='Density scale',
    hover_name='country',
    hover_data=['density'],
    mapbox_style='carto-positron',
    center={'lat': 25, 'lon': 10},
    zoom=0.58,
    opacity=0.75)

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="White",
)

fig.show()